In [2]:
# --- ورود به حساب کاربری Hugging Face ---
# این کد را پس از تنظیم 'HF_TOKEN' در Kaggle Secrets، در یک سلول جداگانه اجرا کنید.
from huggingface_hub import login
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("HF_TOKEN")
try:
    login(token=secret_value)
    print("\nSuccessfully logged in to Hugging Face using Kaggle Secret.")
except:
    print("\nError: HF_TOKEN Kaggle Secret not found. Please ensure it's created and attached.")
    print("You can try manual login by uncommenting the line below and running again.")
    # login() # uncomment this line to try manual login if secret method fails



Successfully logged in to Hugging Face using Kaggle Secret.


In [3]:
# --- مرحله 1: آماده سازی محیط Kaggle و نصب کتابخانه ها ---
# این دستورات را مستقیماً در یک سلول کد در Kaggle Notebook خودتان اجرا کنید.

# --- تنظیمات محیطی برای استفاده از TPU ---
import os
# در محیط TPU، نیازی به تنظیم CUDA_VISIBLE_DEVICES یا PYTORCH_CUDA_ALLOC_CONF نیست.
# TPU ها به صورت متفاوتی مدیریت می شوند.

# برای جلوگیری از هشدارهای مربوط به توکن سازی موازی
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print("Environment configured for TPU usage.")


# نصب کتابخانه های اصلی مورد نیاز برای فاین تیونینگ LLM ها
# transformers: کتابخانه اصلی Hugging Face برای مدل های ترانسفورمر
# accelerate: برای آموزش بهینه و توزیع شده (مهم برای TPU)
# peft: Parameter-Efficient Fine-Tuning (برای LoRA)
# trl: Transformer Reinforcement Learning (شامل SFTTrainer)
# datasets: برای بارگذاری و پردازش دیتاست ها از Hugging Face Hub
# bitsandbytes و unsloth برای TPU استفاده نمی شوند و حذف شده اند.
!pip install -q -U transformers accelerate peft trl scipy datasets
# نصب کتابخانه huggingface_hub برای ابزارهای خط فرمان و لاگین
!pip install -q -U "huggingface_hub[cli]"

print("All necessary libraries are being installed. This might take a few minutes.")
print("Please ensure your Kaggle Notebook has a TPU accelerator enabled (e.g., TPU v3-8).")

# --- ورود به حساب کاربری Hugging Face ---
# این مرحله برای دانلود مدل MedGemma ضروری است.
# توکن خود را از https://huggingface.co/settings/tokens ایجاد کرده و آن را به عنوان یک Secret در Kaggle اضافه کنید.
# نام Secret را 'HF_TOKEN' قرار دهید.


Environment configured for TPU usage.

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
All necessary libraries are being installed. This might take a few minutes.
Please ensure your Kaggle Notebook has a TPU accelerator enabled (e.g., TPU v3-8).


In [4]:
# --- مرحله 2: بارگذاری و آماده سازی دیتاست ---
# این کد را در یک سلول جدید در Kaggle Notebook خودتان اجرا کنید.
# مطمئن شوید که کتابخانه 'datasets' نصب شده و به Hugging Face لاگین کرده اید.

from datasets import load_dataset

print("Loading the gaokerena/MF3QA dataset from Hugging Face Hub...")
try:
    # بارگذاری دیتاست از Hugging Face Hub
    dataset = load_dataset("gaokerena/MF3QA")
    print("Dataset loaded successfully.")
    print("\nDataset structure:")
    print(dataset) # نمایش ساختار دیتاست (معمولاً شامل 'train', 'validation', 'test' split ها)
    
    # ما برای فاین تیونینگ از 'train' split استفاده می کنیم.
    # اگر دیتاست فقط یک split داشته باشد و اسمش 'train' نباشد، باید اسم صحیح را پیدا کرد.
    if 'train' in dataset:
        train_dataset = dataset['train']
    else:
        # اگر فقط یک split وجود دارد، آن را انتخاب کن
        train_dataset = dataset[list(dataset.keys())[0]]

    print(f"\nNumber of examples in the training split: {len(train_dataset)}")
    print("\nFirst 5 raw examples from the dataset:")
    for i in range(min(5, len(train_dataset))):
        print(f"--- Example {i+1} ---")
        # استفاده از 'Question' و 'Answer' با حرف بزرگ (مطابق با ساختار دیتاست MF3QA)
        print(f"Question: {train_dataset[i]['Question']}")
        print(f"Answer: {train_dataset[i]['Answer']}")

    # --- فرمت بندی دیتاست برای Instruction Tuning ---
    # مدل های LLM Instruction-tuned (مثل MedGemma) بهتر است داده ها را به فرمت مکالمه ای دریافت کنند.
    # ما جفت های سوال و جواب را به یک رشته واحد تبدیل می کنیم که مدل بتواند آن را تکمیل کند.
    # مثلاً "سوال: [سوال_متن]\nپاسخ: [پاسخ_متن]"

    def format_example(example):
        # اطمینان حاصل می کنیم که 'Question' و 'Answer' وجود دارند و رشته هستند.
        question = str(example.get('Question', '')).strip()
        answer = str(example.get('Answer', '')).strip()

        # فرمت استاندارد برای Instruction Tuning
        # می توان از <|user|> و <|assistant|> هم استفاده کرد اگر مدل با آن ها آموزش دیده باشد.
        # اما برای شروع، این فرمت ساده و واضح است.
        formatted_text = f"سوال: {question}\nپاسخ: {answer}"
        return {"text": formatted_text}

    print("\nFormatting the dataset into 'text' column...")
    # متد .map() را برای اعمال تابع format_example روی کل دیتاست استفاده می کنیم.
    # remove_columns=train_dataset.column_names: ستون های اصلی (Question, Answer, Source) را حذف کرده و فقط ستون 'text' را نگه می دارد.
    formatted_dataset = train_dataset.map(format_example, remove_columns=train_dataset.column_names)
    
    print("\nFirst 3 formatted examples:")
    for i in range(min(3, len(formatted_dataset))):
        print(f"--- Formatted Example {i+1} ---")
        print(formatted_dataset[i]['text'])

    # دیتاست آماده شده را می توانیم برای مراحل بعدی (توکن سازی و آموزش) استفاده کنیم.
    # نیازی به ذخیره به دیسک در این مرحله نیست و می توانیم آن را در حافظه نگه داریم.
    
    print("\nDataset preparation for fine-tuning is complete. Ready for model loading and tokenization.")

except Exception as e:
    print(f"An error occurred during dataset loading or preparation: {e}")
    print("Please ensure you have successfully logged in to Hugging Face and your internet connection is stable.")


Loading the gaokerena/MF3QA dataset from Hugging Face Hub...
Dataset loaded successfully.

Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer', 'Source'],
        num_rows: 20000
    })
    dev: Dataset({
        features: ['Question', 'Answer', 'Source'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['Question', 'Answer', 'Source'],
        num_rows: 2000
    })
})

Number of examples in the training split: 20000

First 5 raw examples from the dataset:
--- Example 1 ---
Question: ۱ماهپیش از خواب پریدم از ترس شدید بعداز اون ترس،فشارم بالا میرفت ۱۴ ۱۵ با سر درد و درد قفسه سینه همراه بود با پرانول کنترش میکردم ولی الان ۲ ۳ روزه فشارم میاد پاین ۸ ۹ روی ۳ ۴ وقتایی که میرم پیاده روی یا کلا فعالیتی دارم بعدش ضربانم تا ۶ ۷ ساعت حتی بیشتر تنده و کند و طبیعی نمیشه خستهه شدم دیگهمیترسم اتفاقی برام بیوفته فشار ۸ ۹ روری ۴ ۵ خطرناکه؟
Answer: تجربه‌ای که شرح داده‌اید، نشان‌دهنده‌ی رخدادهای فیزیولوژیک و احتمالاً پاتولوژیک در بدن شما است

In [5]:
# --- مرحله 3: بارگذاری مدل پایه و توکنایزر ---
# این کد را در یک سلول جدید در Kaggle Notebook خودتان اجرا کنید.

# مطمئن شوید که کتابخانه های لازم (transformers, accelerate, peft, trl, datasets) نصب شده اند.
# همچنین مطمئن شوید که به Hugging Face لاگین کرده اید.

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
# در محیط TPU، نیازی به BitsAndBytesConfig و unsloth نیست، زیرا اینها عمدتاً برای بهینه سازی GPU هستند.
# from bitsandbytes import BitsAndBytesConfig # حذف شد
# from unsloth import FastLanguageModel # حذف شد
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
# from trl import SFTTrainer # *** اصلاح شده: SFTTrainer از اینجا حذف شد و به مرحله بعد منتقل می شود ***
from datasets import Dataset # برای اطمینان از دسترسی به کلاس Dataset

# در محیط TPU، USE_UNSLOTH همیشه False خواهد بود.
USE_UNSLOTH = False

# نام مدل MedGemma که می‌خواهیم فاین تیون کنیم.
# توصیه می‌شود با 4B شروع کنید.
model_name = "google/medgemma-4b-it" # یا "google/medgemma-27b"

# --- بارگذاری توکنایزر ---
print(f"Loading tokenizer for model: {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
# اطمینان از تنظیم pad_token برای توکنایزر
# این برای مدل های decoder-only مهم است.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token # End-of-sequence token به عنوان pad_token

print("Tokenizer loaded successfully.")

# --- بارگذاری مدل ---
print(f"Loading model: {model_name}...")
# برای TPU، مدل را بدون تنظیمات BitsAndBytesConfig بارگذاری می کنیم.
# torch_dtype را روی torch.bfloat16 یا torch.float32 تنظیم می کنیم.
# bfloat16 برای TPU ها بهینه است و توصیه می شود.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16, # استفاده از bfloat16 برای TPU ها
    device_map="auto", # برای توزیع مدل روی دستگاه های موجود (TPU cores)
)
print("Model loaded successfully.")

# --- تنظیمات توکنایزر برای آموزش ---
# این تنظیمات برای اطمینان از اینکه توکنایزر به درستی برای آموزش آماده است، ضروری است.
tokenizer.padding_side = "right" # پدینگ از سمت راست (برای مدل های decoder-only توصیه می شود)

print(f"\nModel '{model_name}' and Tokenizer loaded successfully for TPU.")
print("\nModel structure (first few layers):")
print(model) # نمایش ساختار مدل

print("\nTokenizer padding side set to 'right'.")
print("Ready for LoRA configuration and Trainer setup.")


/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:251: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
E0000 00:00:1753121524.684954    5177 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:230


Loading tokenizer for model: google/medgemma-4b-it...
Tokenizer loaded successfully.
Loading model: google/medgemma-4b-it...


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.95it/s]


Model loaded successfully.

Model 'google/medgemma-4b-it' and Tokenizer loaded successfully for TPU.

Model structure (first few layers):
Gemma3ForConditionalGeneration(
  (model): Gemma3Model(
    (vision_tower): SiglipVisionModel(
      (vision_model): SiglipVisionTransformer(
        (embeddings): SiglipVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
          (position_embedding): Embedding(4096, 1152)
        )
        (encoder): SiglipEncoder(
          (layers): ModuleList(
            (0-26): 27 x SiglipEncoderLayer(
              (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
              (self_attn): SiglipAttention(
                (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (out_proj): L

In [ ]:
# --- مرحله 4: پیکربندی LoRA و Trainer ---
# این کد را در یک سلول جدید در Kaggle Notebook خودتان اجرا کنید.

# مطمئن شوید که 'model' و 'tokenizer' از مرحله قبل در دسترس هستند.
# همچنین 'formatted_dataset' که در مرحله 2 آماده کردیم.

from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments # TrainingArguments همچنان برای برخی لاگ ها و سازگاری لازم است، اما پارامترهای آن به SFTConfig منتقل می شود
from trl import SFTTrainer, SFTConfig # *** SFTConfig اضافه شد ***

print("Configuring LoRA and Training Arguments for TPU...")

# --- 4.1: پیکربندی LoRA (Low-Rank Adaptation) ---
# LoRA به ما اجازه می دهد تا مدل را با آموزش فقط بخش کوچکی از وزن های آن فاین تیون کنیم.
# این کار مصرف حافظه و زمان آموزش را به شدت کاهش می دهد.
lora_config = LoraConfig(
    r=16, # r (rank): ابعاد ماتریس های LoRA. مقادیر رایج 8، 16، 32، 64 هستند.
          # مقدار بالاتر = قدرت بیانی بیشتر اما مصرف حافظه و زمان بیشتر. 16 یک نقطه شروع خوب است.
    lora_alpha=16, # lora_alpha: یک فاکتور مقیاس گذاری برای وزن های LoRA. معمولاً برابر با r یا دو برابر آن تنظیم می شود.
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
          # target_modules: لایه هایی در مدل که LoRA روی آنها اعمال می شود.
          # اینها معمولاً لایه های خطی در بلوک های ترانسفورمر هستند که در Gemma/MedGemma رایج هستند.
    lora_dropout=0.05, # lora_dropout: نرخ دراپ اوت برای لایه های LoRA برای جلوگیری از Overfitting.
    bias="none", # bias: نحوه مدیریت بایاس در لایه های LoRA. "none" معمولاً برای LLM ها خوب است.
    task_type="CAUSAL_LM", # task_type: نوع وظیفه مدل. CAUSAL_LM برای مدل های تولید متن (مانند Gemma/MedGemma) است.
)

# --- اعمال پیکربندی LoRA به مدل ---
# برای TPU، ما به صورت دستی آداپتور LoRA را به مدل اضافه می کنیم.
model = get_peft_model(model, lora_config)

print("LoRA adapters attached to the model.")

# --- 4.2: ایجاد SFTConfig (جایگزین TrainingArguments برای SFTTrainer) ---
# SFTConfig تمام پارامترهای آموزش و پارامترهای خاص SFTTrainer را در یک شیء واحد نگه می دارد.
sft_config = SFTConfig(
    output_dir="./results", # دایرکتوری برای ذخیره لاگ ها، چک پوینت ها و مدل نهایی
    num_train_epochs=5, # تعداد اپوک ها (دوره ها) برای آموزش. 3-5 اپوک برای شروع خوب است.
    per_device_train_batch_size=8, # بچ سایز برای TPU معمولاً می تواند بزرگتر باشد (با 8 هسته TPU).
                                   # اگر OOM گرفتید، آن را کاهش دهید (مثلاً 4).
    gradient_accumulation_steps=1, # در TPU با بچ سایز بزرگتر، معمولاً نیازی به accumulation نیست.
    optim="adamw_torch", # بهینه ساز (optimizer) مورد استفاده. adamw_torch برای TPU ها مناسب است.
    save_steps=500, # تعداد گام ها (steps) برای ذخیره چک پوینت مدل.
    logging_steps=50, # تعداد گام ها برای لاگ کردن اطلاعات آموزش (loss و غیره).
    learning_rate=2e-4, # learning_rate: نرخ یادگیری برای بهینه ساز.
    fp16=False, # در TPU از bfloat16 استفاده می کنیم، پس fp16 را False نگه می داریم.
    bf16=True, # *** فعال کردن bfloat16 برای TPU ***
    max_grad_norm=0.3, # حداکثر نرم گرادیان برای جلوگیری از انفجار گرادیان.
    warmup_ratio=0.03, # نسبت گام های گرم کردن (warmup) در ابتدای آموزش.
    lr_scheduler_type="constant", # نوع زمانبندی نرخ یادگیری (learning rate scheduler).
    report_to="tensorboard", # ابزاری برای گزارش دهی (می توانید "none" یا "wandb" را هم استفاده کنید).
    disable_tqdm=False, # غیرفعال کردن نوار پیشرفت tqdm در طول آموزش (اختیاری).
    
    # پارامترهای خاص SFTTrainer که اکنون در SFTConfig قرار می گیرند:
    max_seq_length=2048, # حداکثر طول دنباله. این باید با max_seq_length توکنایزر شما مطابقت داشته باشد.
    dataset_text_field="text", # نام ستونی در دیتاست که متن فرمت شده برای آموزش را شامل می شود.
    packing=False, # برای ترکیب چندین نمونه کوتاه در یک دنباله طولانی تر.
                   # برای دیتاست Q&A شما که پاسخ ها کوتاه هستند، ممکن است مفید باشد.
                   # اما برای شروع False نگه می داریم تا پیچیدگی کمتر شود.
    # اگر می خواهید مدل را در Hugging Face Hub آپلود کنید، این پارامترها را در SFTConfig فعال کنید:
    # push_to_hub=True,
    # hub_model_id="your-username/medgemma-fa-medical-qa", # نام مخزن در Hugging Face Hub
    # hub_private_repo=False, # اگر مخزن خصوصی است، True کنید.
    # hub_strategy="every_save", # هر بار که مدل ذخیره می شود، به هاب هم push شود.
)

# --- 4.3: آماده سازی SFTTrainer ---
# SFTTrainer (Supervised Fine-Tuning Trainer) از کتابخانه trl برای فاین تیونینگ مدل های زبان استفاده می شود.
# این Trainer تمامی مراحل آموزش را مدیریت می کند.

trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    peft_config=lora_config,
    args=sft_config,
    processing_class=tokenizer  # ← جایگزین `tokenizer=...`
)


print("\nLoRA and Training Arguments configured successfully.")
print("SFTTrainer initialized. Ready to start training!")


Configuring LoRA and Training Arguments for TPU...


LoRA adapters attached to the model.


E0000 00:00:1753121535.677118    5177 common_lib.cc:621] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:232


In [ ]:
import trl
print(f"trl version: {trl.__version__}")

In [ ]:
!git config --global user.name "lbehradl"
!git clone https://github.com/unslothai/unsloth.git
!pip install -q -U ./unsloth

In [ ]:
!pip install -q -U unsloth_zoo


In [ ]:
import unsloth
try:
    from unsloth import FastLanguageModel
    print("Unsloth detected. Using FastLanguageModel for optimized loading.")
    USE_UNSLOTH = True
except ImportError:
    print("Unsloth not found. Falling back to standard Hugging Face loading.")
    USE_UNSLOTH = False


In [ ]:
# --- مرحله 3: بارگذاری مدل پایه و توکنایزر ---
# این کد را در یک سلول جدید در Kaggle Notebook خودتان اجرا کنید.

# --- نصب مجدد کتابخانه های مورد نیاز برای اطمینان از وجود 'trl' و سایرین ---
# این دستورات در ابتدای این سلول قرار داده شده اند تا اطمینان حاصل شود که همه پیش نیازها قبل از import فراهم هستند.
print("Ensuring all necessary libraries are installed...")
!pip install -q -U transformers accelerate peft trl bitsandbytes scipy datasets
!pip install -q -U "huggingface_hub[cli]"
!pip install -q -U git+https://github.com/unsloth/unsloth.git
print("Libraries installation/update complete.")

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import Dataset # برای اطمینان از دسترسی به کلاس Dataset

# اگر از Unsloth استفاده می‌کنید، بهتر است از توابع بارگذاری آن استفاده کنید.
# این کار بهینه‌سازی‌های خاص Unsloth را فعال می‌کند.
try:
    from unsloth import FastLanguageModel
    print("Unsloth detected. Using FastLanguageModel for optimized loading.")
    USE_UNSLOTH = True
except ImportError:
    print("Unsloth not found. Falling back to standard Hugging Face loading.")
    USE_UNSLOTH = False

# نام مدل MedGemma که می‌خواهیم فاین تیون کنیم.
# توصیه می‌شود با 4B شروع کنید مگر اینکه GPU قدرتمند (A100) داشته باشید.
model_name = "google/medgemma-4b-it" # یا "google/medgemma-27b"

# --- تنظیمات کوانتیزیشن (Quantization) ---
# برای کاهش مصرف حافظه GPU، مدل را به 4-bit کوانتیزه می کنیم (QLoRA).
# این کار به شما امکان می دهد مدل های بزرگتر را روی GPU های با RAM کمتر اجرا کنید.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", # نوع کوانتیزیشن
    bnb_4bit_compute_dtype=torch.bfloat16, # نوع داده برای محاسبات
    bnb_4bit_use_double_quant=True, # استفاده از کوانتیزیشن دوگانه برای کاهش بیشتر حافظه
)

# --- بارگذاری مدل و توکنایزر ---
# اگر Unsloth در دسترس باشد، از آن برای بارگذاری بهینه استفاده می‌کنیم.
if USE_UNSLOTH:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=2048, # حداکثر طول دنباله (sequence length) برای ورودی های مدل
                               # این مقدار را می توانید بر اساس طول سوالات و پاسخ های خود تنظیم کنید.
                               # 2048 یک مقدار رایج و مناسب برای شروع است.
        dtype=None, # None به unsloth اجازه می دهد بهترین dtype را انتخاب کند (معمولا bfloat16)
        load_in_4bit=True, # فعال کردن کوانتیزیشن 4-bit
    )
else:
    # بارگذاری توکنایزر
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # اطمینان از تنظیم pad_token برای توکنایزر
    # این برای مدل های decoder-only مهم است.
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token # End-of-sequence token به عنوان pad_token

    # بارگذاری مدل با تنظیمات کوانتیزیشن
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto", # برای توزیع مدل روی GPU های موجود
        torch_dtype=torch.bfloat16, # استفاده از bfloat16 برای محاسبات
    )
    # آماده سازی مدل برای آموزش LoRA با کوانتیزیشن 4-bit
    model = prepare_model_for_kbit_training(model)

print(f"\nModel '{model_name}' and Tokenizer loaded successfully.")
print("\nModel structure (first few layers):")
print(model) # نمایش ساختار مدل

# --- تنظیمات توکنایزر برای آموزش ---
# این تنظیمات برای اطمینان از اینکه توکنایزر به درستی برای آموزش آماده است، ضروری است.
tokenizer.padding_side = "right" # پدینگ از سمت راست (برای مدل های decoder-only توصیه می شود)

print("\nTokenizer padding side set to 'right'.")
print("Ready for LoRA configuration and training.")


In [ ]:
# --- مرحله 4: پیکربندی LoRA و Trainer ---
# این کد را در یک سلول جدید در Kaggle Notebook خودتان اجرا کنید.

# مطمئن شوید که 'model' و 'tokenizer' از مرحله قبل در دسترس هستند.
# همچنین 'formatted_dataset' که در مرحله 2 آماده کردیم.

from peft import LoraConfig, get_peft_model # get_peft_model اضافه شد
from transformers import TrainingArguments
from trl import SFTTrainer

print("Configuring LoRA and Training Arguments...")

# --- 4.1: پیکربندی LoRA (Low-Rank Adaptation) ---
# LoRA به ما اجازه می دهد تا مدل را با آموزش فقط بخش کوچکی از وزن های آن فاین تیون کنیم.
# این کار مصرف حافظه و زمان آموزش را به شدت کاهش می دهد.
lora_config = LoraConfig(
    r=16, # r (rank): ابعاد ماتریس های LoRA. مقادیر رایج 8، 16، 32، 64 هستند.
          # مقدار بالاتر = قدرت بیانی بیشتر اما مصرف حافظه و زمان بیشتر. 16 یک نقطه شروع خوب است.
    lora_alpha=16, # lora_alpha: یک فاکتور مقیاس گذاری برای وزن های LoRA. معمولاً برابر با r یا دو برابر آن تنظیم می شود.
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
          # target_modules: لایه هایی در مدل که LoRA روی آنها اعمال می شود.
          # اینها معمولاً لایه های خطی در بلوک های ترانسفورمر هستند که در Gemma/MedGemma رایج هستند.
    lora_dropout=0.05, # lora_dropout: نرخ دراپ اوت برای لایه های LoRA برای جلوگیری از Overfitting.
    bias="none", # bias: نحوه مدیریت بایاس در لایه های LoRA. "none" معمولاً برای LLM ها خوب است.
    task_type="CAUSAL_LM", # task_type: نوع وظیفه مدل. CAUSAL_LM برای مدل های تولید متن (مانند Gemma/MedGemma) است.
)

# --- اضافه شده: اعمال پیکربندی LoRA به مدل ---
# این مرحله مدل پایه را با لایه های LoRA "تقویت" می کند و آن را برای فاین تیونینگ آماده می سازد.
# این کار باید قبل از پاس دادن مدل به SFTTrainer انجام شود.
model.add_adapter(lora_config) # اگر از Unsloth استفاده می کنید، FastLanguageModel.from_pretrained
                               # ممکن است این مرحله را به صورت خودکار انجام دهد.
                               # اما برای اطمینان و سازگاری با حالت استاندارد، این خط اضافه می شود.
                               # اگر Unsloth از قبل آداپتور را اضافه کرده باشد، این خط ممکن است
                               # هشدار دهد یا به سادگی کاری انجام ندهد.
# model = get_peft_model(model, lora_config) # این خط برای حالت استاندارد Hugging Face است.
                                           # Unsloth معمولا این کار را در FastLanguageModel.from_pretrained انجام می دهد.
                                           # اگر از Unsloth استفاده می کنید، نیازی به فراخوانی get_peft_model نیست.
                                           # اگر Unsloth را حذف کردید، این خط را فعال کنید.

# --- 4.2: تنظیمات آموزش (TrainingArguments) ---
# این تنظیمات رفتار Trainer را در طول آموزش کنترل می کند.
training_arguments = TrainingArguments(
    output_dir="./results", # دایرکتوری برای ذخیره لاگ ها، چک پوینت ها و مدل نهایی
    num_train_epochs=3, # تعداد اپوک ها (دوره ها) برای آموزش. 3-5 اپوک برای شروع خوب است.
                        # با توجه به 15 هزار نمونه شما و 24 هزار نمونه دیتاست MF3QA، این عدد مناسب است.
    per_device_train_batch_size=4, # تعداد نمونه در هر بچ (batch) برای هر GPU.
                                   # این مقدار به RAM GPU شما بستگی دارد. اگر OOM گرفتید، آن را کاهش دهید (مثلاً 2).
    gradient_accumulation_steps=2, # تعداد گام های انباشت گرادیان.
                                   # گرادیان ها را در چندین بچ کوچک جمع می کند تا به یک بچ بزرگتر برسد.
                                   # این به شبیه سازی batch size بزرگتر کمک می کند بدون مصرف زیاد RAM.
                                   # (مثلاً با batch_size=4 و gradient_accumulation_steps=2، batch size موثر 8 می شود)
    optim="paged_adamw_8bit", # بهینه ساز (optimizer) مورد استفاده. paged_adamw_8bit برای QLoRA بهینه است.
    save_steps=500, # تعداد گام ها (steps) برای ذخیره چک پوینت مدل.
    logging_steps=50, # تعداد گام ها برای لاگ کردن اطلاعات آموزش (loss و غیره).
    learning_rate=2e-4, # learning_rate: نرخ یادگیری برای بهینه ساز.
    fp16=False, # *** اصلاح شده: استفاده از float32 (با تنظیم هر دو fp16 و bf16 به False) ***
    bf16=False, # *** اصلاح شده: استفاده از float32 (با تنظیم هر دو fp16 و bf16 به False) ***
    max_grad_norm=0.3, # حداکثر نرم گرادیان برای جلوگیری از انفجار گرادیان.
    warmup_ratio=0.03, # نسبت گام های گرم کردن (warmup) در ابتدای آموزش.
    lr_scheduler_type="constant", # نوع زمانبندی نرخ یادگیری (learning rate scheduler).
    report_to="tensorboard", # ابزاری برای گزارش دهی (می توانید "none" یا "wandb" را هم استفاده کنید).
    disable_tqdm=True, # غیرفعال کردن نوار پیشرفت tqdm در طول آموزش (اختیاری).
    # اگر می خواهید مدل را در Hugging Face Hub آپلود کنید، این پارامترها را فعال کنید:
    # push_to_hub=True,
    # hub_model_id="your-username/medgemma-fa-medical-qa", # نام مخزن در Hugging Face Hub
    # hub_private_repo=False, # اگر مخزن خصوصی است، True کنید.
    # hub_strategy="every_save", # هر بار که مدل ذخیره می شود، به هاب هم push شود.
)

# --- 4.3: آماده سازی SFTTrainer ---
# SFTTrainer (Supervised Fine-Tuning Trainer) از کتابخانه trl برای فاین تیونینگ مدل های زبان استفاده می شود.
# این Trainer تمامی مراحل آموزش را مدیریت می کند.
trainer = SFTTrainer(
    model=model, # مدل MedGemma که بارگذاری کردیم (حالا با آداپتورهای LoRA متصل شده)
    train_dataset=formatted_dataset, # دیتاست آماده شده فارسی
    peft_config=lora_config, # پیکربندی LoRA
    tokenizer=tokenizer, # توکنایزر مدل
    args=training_arguments, # تنظیمات آموزش
    packing=False, # packing: برای ترکیب چندین نمونه کوتاه در یک دنباله طولانی تر.
                   # برای دیتاست Q&A شما که پاسخ ها کوتاه هستند، ممکن است مفید باشد.
                   # اما برای شروع False نگه می داریم تا پیچیدگی کمتر شود.
    dataset_text_field="text", # نام ستونی در دیتاست که متن فرمت شده برای آموزش را شامل می شود.
)

print("\nLoRA and Training Arguments configured successfully.")
print("SFTTrainer initialized. Ready to start training!")


In [ ]:
# --- مرحله 4: پیکربندی LoRA و Trainer ---
# این کد را در یک سلول جدید در Kaggle Notebook خودتان اجرا کنید.

# مطمئن شوید که 'model' و 'tokenizer' از مرحله قبل در دسترس هستند.
# همچنین 'formatted_dataset' که در مرحله 2 آماده کردیم.

from peft import LoraConfig, get_peft_model # get_peft_model اضافه شد
from transformers import TrainingArguments
from trl import SFTTrainer

print("Configuring LoRA and Training Arguments...")

# --- 4.1: پیکربندی LoRA (Low-Rank Adaptation) ---
# LoRA به ما اجازه می دهد تا مدل را با آموزش فقط بخش کوچکی از وزن های آن فاین تیون کنیم.
# این کار مصرف حافظه و زمان آموزش را به شدت کاهش می دهد.
lora_config = LoraConfig(
    r=16, # r (rank): ابعاد ماتریس های LoRA. مقادیم رایج 8، 16، 32، 64 هستند.
          # مقدار بالاتر = قدرت بیانی بیشتر اما مصرف حافظه و زمان بیشتر. 16 یک نقطه شروع خوب است.
    lora_alpha=16, # lora_alpha: یک فاکتور مقیاس گذاری برای وزن های LoRA. معمولاً برابر با r یا دو برابر آن تنظیم می شود.
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
          # target_modules: لایه هایی در مدل که LoRA روی آنها اعمال می شود.
          # اینها معمولاً لایه های خطی در بلوک های ترانسفورمر هستند که در Gemma/MedGemma رایج هستند.
    lora_dropout=0.05, # lora_dropout: نرخ دراپ اوت برای لایه های LoRA برای جلوگیری از Overfitting.
    bias="none", # bias: نحوه مدیریت بایاس در لایه های LoRA. "none" معمولاً برای LLM ها خوب است.
    task_type="CAUSAL_LM", # task_type: نوع وظیفه مدل. CAUSAL_LM برای مدل های تولید متن (مانند Gemma/MedGemma) است.
)

# --- اصلاح شده: اعمال پیکربندی LoRA به مدل (خط model.add_adapter حذف شد) ---
# Unsloth (FastLanguageModel.from_pretrained) به صورت خودکار آداپتور LoRA را اضافه می کند.
# بنابراین نیازی به فراخوانی دستی model.add_adapter یا get_peft_model نیست.


# --- 4.2: تنظیمات آموزش (TrainingArguments) ---
# این تنظیمات رفتار Trainer را در طول آموزش کنترل می کند.
training_arguments = TrainingArguments(
    output_dir="./results", # دایرکتوری برای ذخیره لاگ ها، چک پوینت ها و مدل نهایی
    num_train_epochs=5, # تعداد اپوک ها (دوره ها) برای آموزش. 3-5 اپوک برای شروع خوب است.
                        # با توجه به 15 هزار نمونه شما و 24 هزار نمونه دیتاست MF3QA، این عدد مناسب است.
    per_device_train_batch_size=2, # *** اصلاح شده: کاهش بچ سایز برای کاهش مصرف حافظه GPU ***
    gradient_accumulation_steps=4, # *** اصلاح شده: افزایش گام های انباشت گرادیان برای حفظ effective batch size ***
                                   # (با بچ سایز 2 و انباشت 4، effective batch size همچنان 8 است)
    optim="paged_adamw_8bit", # بهینه ساز (optimizer) مورد استفاده. paged_adamw_8bit برای QLoRA بهینه است.
    save_steps=500, # تعداد گام ها (steps) برای ذخیره چک پوینت مدل.
    logging_steps=50, # تعداد گام ها برای لاگ کردن اطلاعات آموزش (loss و غیره).
    learning_rate=2e-4, # learning_rate: نرخ یادگیری برای بهینه ساز.
    fp16=False, # استفاده از float32 (با تنظیم هر دو fp16 و bf16 به False)
    bf16=False, # استفاده از float32 (با تنظیم هر دو fp16 و bf16 به False)
    max_grad_norm=0.3, # حداکثر نرم گرادیان برای جلوگیری از انفجار گرادیان.
    warmup_ratio=0.03, # نسبت گام های گرم کردن (warmup) در ابتدای آموزش.
    lr_scheduler_type="constant", # نوع زمانبندی نرخ یادگیری (learning rate scheduler).
    report_to="tensorboard", # ابزاری برای گزارش دهی (می توانید "none" یا "wandb" را هم استفاده کنید).
    disable_tqdm=True, # غیرفعال کردن نوار پیشرفت tqdm در طول آموزش (اختیاری).
    # اگر می خواهید مدل را در Hugging Face Hub آپلود کنید، این پارامترها را فعال کنید:
    # push_to_hub=True,
    # hub_model_id="your-username/medgemma-fa-medical-qa", # نام مخزن در Hugging Face Hub
    # hub_private_repo=False, # اگر مخزن خصوصی است، True کنید.
    # hub_strategy="every_save", # هر بار که مدل ذخیره می شود، به هاب هم push شود.
)

# --- 4.3: آماده سازی SFTTrainer ---
# SFTTrainer (Supervised Fine-Tuning Trainer) از کتابخانه trl برای فاین تیونینگ مدل های زبان استفاده می شود.
# این Trainer تمامی مراحل آموزش را مدیریت می کند.
trainer = SFTTrainer(
    model=model, # مدل MedGemma که بارگذاری کردیم (حالا با آداپتورهای LoRA متصل شده)
    train_dataset=formatted_dataset, # دیتاست آماده شده فارسی
    peft_config=lora_config, # پیکربندی LoRA
    tokenizer=tokenizer, # توکنایزر مدل
    args=training_arguments, # تنظیمات آموزش
    packing=False, # packing: برای ترکیب چندین نمونه کوتاه در یک دنباله طولانی تر.
                   # برای دیتاست Q&A شما که پاسخ ها کوتاه هستند، ممکن است مفید باشد.
                   # اما برای شروع False نگه می داریم تا پیچیدگی کمتر شود.
    dataset_text_field="text", # نام ستونی در دیتاست که متن فرمت شده برای آموزش را شامل می شود.
)

print("\nLoRA and Training Arguments configured successfully.")
print("SFTTrainer initialized. Ready to start training!")
